## <span style="color:grey">Initialization</span>

In [ ]:
try:
    mode
except Exception as e:
    %run "KiteConnect_Library.ipynb"  
    logging.debug(e)
    toDate = dt.datetime(2019,4,18,15,15)
    fromDate = toDate - dt.timedelta(days = 1, hours=4)
    mode = "Preanalysis"
    papertrade = True


##  <span style="color:green">Trading Strategy </span>

In [ ]:
%run "KiteConnect_Strategies_v2.ipynb"  

## Batch Analysis - Multithreaded

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objs as go
from plotly import tools
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

noofcandles = 50
threaded = True
from multiprocessing import Pool
def threadedBatchAnalysis(x):
    global fig
    #fig = ""
    temp_data = getData(x, fromDate=toDate - dt.timedelta(days=365), toDate=toDate, freq="day")
    #logging.debug(temp_data)

    fig = createPlot(x)
    
    plot_set1(temp_data)
    
    a = algoTrade(temp_data)
    buy, sell = a.tradeDecision()
    buy_df = annotateBuySell(buy, "Buy")
    sell_df = annotateBuySell(sell, "Sell")
    #print(os.getcwd())
    plotData(x, False)

if __name__ == '__main__':
    portfolio = pd.DataFrame(['TCS', 'RIIL', 'HDFC','BAJAJ-AUTO'])
    multithreading = False
    if multithreading:
        p = Pool(10)
        p.map_async(threadedBatchAnalysis, portfolio[0])
        #p.apply_async(threadedBatchAnalysis, ['SBIN'])
        #portfolio
    else:
        for index, row in portfolio.iterrows():
            threadedBatchAnalysis(row[0])

## Batch Downloader for stock data

In [ ]:
stock_list = pd.read_csv('data/zerodha_margin.csv')

stock_list.drop(['Serial No.','Unnamed: 3','Serial No..1','Stocks not allowed for MIS','Unnamed: 6', 'Reason', 'Stocks which were allowed but temporarily banned', 'Serial No..2'], axis=1, inplace=True)



stock_list.dropna(inplace=True)

stock_list.columns = ['symbol', 'margin']

stock_list['symbol'] = stock_list['symbol'].str.replace(':EQ','')

stock_list['margin'] = stock_list['margin'].str.replace('X','')

stock_list = stock_list.sort_values(['margin','symbol'])

portfolio = stock_list['symbol']

In [ ]:
import time
#logger.info("Resume download")
toDate = dt.datetime.now() #-dt.timedelta(days=2*365)
fromDate = toDate - dt.timedelta(days=2*365)
freq = 'minute'
counter = 0
#portfolio = ['NIFTY SMALL 100','NIFTY INFRA','NIFTY MEDIA','NIFTY MNC','NIFTY 200','NIFTY FIN SERVICE','NIFTY PHARMA','NIFTY COMMODITIES','NIFTY REALTY','NIFTY PSU BANK','NIFTY BANK','NIFTY IT','NIFTY 50','NIFTY AUTO','NIFTY 100','NIFTY METAL','NIFTY JUNIOR','NIFTY ENERGY','NIFTY MIDCAP 50','NDIA VIX','NIFTY MIDCAP 100','NIFTY FMCG']
portfolio = ['NIFTY19MAYFUT']
for symbol in portfolio:
    try:
        logger.debug('Dowloading data for:'+symbol)
        getData(symbol, fromDate, toDate, exchange="NFO", freq=freq, force=False, symbolToken='13427458')
    except:
        logger.debug('Could not download: '+symbol)
        time.sleep(2)
        
    gc.collect()

## Futures Download - NIFTY

In [ ]:
futures_df = getInstruments('NFO')

futures_df[futures_df.index=='NIFTY19MAYFUT']

toDate = dt.datetime.now() #-dt.timedelta(days=2*365)
fromDate = toDate - dt.timedelta(days=2*365)
freq = 'minute'
symbolToken = 13427458


raw_data = pd.DataFrame(data=kite.historical_data(symbolToken, fromDate, toDate, freq, continuous=False))
raw_data = raw_data.set_index('date').tz_localize(None)

exchange = 'NFO'
key = freq+"/"+exchange+"/"+symbol
raw_data.to_hdf("kite_data/kite_cache.h5", key=key, mode="a", format="table")